In [14]:
# 1 重要
# 2 KNN CNN 2种
# 3 样本 
# 4 旧瓶装新酒 ：数字识别的不同
# 4.1 网络 4。2 每一级 4.3 先原理 后代码 
# 本质：knn test 样本 K个 max4 3个1 -》1

# step1 load Data  1.1 随机数 1.2 4组 训练 测试 （图片 和 标签）
# step2 knn test train distance 5*500 = 2500一共有2500个距离，可能情况 784=28*28
# step3 knn k个最近的图片5 500 1-》500train （找出最接近的4张）
# 4 k个最近的图片-> parse centent label
# 5 label -》 数字 p9 测试图片-》数据
# 6 检测概率统计
import tensorflow as tf
import numpy as np
import random # 生成随机的数组进行测试
# 调用教程
from tensorflow.examples.tutorials.mnist import input_data
# load data 2 one_hot : 1 0000 参数1 fileName ； one_hot，有一个内容为1 ，其他都为0
# one_hot：一个为1，其他为0
# 全部装载进mnist
mnist = input_data.read_data_sets('MNIST_data',one_hot=True)
# 属性设置
trainNum = 55000
testNum = 10000#一万张
trainSize = 500#训练时图片
testSize = 5
k = 4 # 四种情况，找到四个最为接近的图片
# data 分解 1 生成trainSize个随机数  ； 2范围0-trainNum； 3 replace=False 是否可以重复：不可重复
trainIndex = np.random.choice(trainNum,trainSize,replace=False)
testIndex = np.random.choice(testNum,testSize,replace=False)
# 上两步，下标生成
trainData = mnist.train.images[trainIndex]# 训练图片
trainLabel = mnist.train.labels[trainIndex]# 训练标签
# 得到测试数据
testData = mnist.test.images[testIndex]# 测试下标：testIndex
testLabel = mnist.test.labels[testIndex]
# 宽28*高28 = 784，描述了图片上所有的像素点
# 打印出来训练数据，与标签数据是否一致，看看结果是否一样
print('trainData.shape=',trainData.shape)#500*784 1行数 图片个数； 2 784?
print('trainLabel.shape=',trainLabel.shape)#500*10
print('testData.shape=',testData.shape)#5*784
print('testLabel.shape=',testLabel.shape)#5*10
print('testLabel=',testLabel)# 4 :testData [0]  3:testData[1] 6 
# tf input  784->image，训练数据和测试数据生成好了
# placeholder：完成数据的加载，每一个784的数据表明一个完整的图片
trainDataInput = tf.placeholder(shape=[None,784],dtype=tf.float32)
trainLabelInput = tf.placeholder(shape=[None,10],dtype=tf.float32)
testDataInput = tf.placeholder(shape=[None,784],dtype=tf.float32)
testLabelInput = tf.placeholder(shape=[None,10],dtype=tf.float32)
# 以上，数据准备好了
 
# 使用KNN具体计算
#knn distance距离公式 原先5*785.-》转换后  5*1*784； 维度转换原因，便于计算
# 测试图片与训练图片的差值的计算
# 测试图片：5 训练：500 ； 每个图片：784 (3D) 2500*784
f1 = tf.expand_dims(testDataInput,1) # 维度转换，维度扩展
f2 = tf.subtract(trainDataInput,f1)# 784 sum(784) # 对应元素相减
f3 = tf.reduce_sum(tf.abs(f2),reduction_indices=2)# 完成784维数据累加 784 abs； reduction_indices设置维度，在第二维度累加
# 5*500 
f4 = tf.negative(f3)# 取反
f5,f6 = tf.nn.top_k(f4,k=4) # 选取f4 最大的四个值
# 等价于选取f3 最小的四个值，分别描述值的内容，以及值的下标
# f6 index->trainLabelInput，最近的四个点的下标
f7 = tf.gather(trainLabelInput,f6)# 将所有的标签获取到
# f8 num reduce_sum  reduction_indices=1 '竖直'，数值方向上维度的累加

# F8 , F9 完成数据的获取，完成数据的累加
f8 = tf.reduce_sum(f7,reduction_indices=1)# 当前维度设置为1
# tf.argmax 选取在某一个最大的值 记录index下标
f9 = tf.argmax(f8,dimension=1)# 测试数据，如果完全相同，就是百分之百
# f9 -> test5 image -> 5 num
with tf.Session() as sess:
    # f1 <- testData 5张图片，待检测
    p1 = sess.run(f1,feed_dict={testDataInput:testData[0:5]})#总共给5张图片
    print('p1=',p1.shape)# p1= (5, 1, 784)，每一个图片用784来表示
    p2 = sess.run(f2,feed_dict={trainDataInput:trainData,testDataInput:testData[0:5]})
    print('p2=',p2.shape)#p2= (5, 500, 784) (1,100) 随便取（1， 100） 这样的点表示第二张测试图片和第一101张的训练图片的距离，距离放到784维中
    p3 = sess.run(f3,feed_dict={trainDataInput:trainData,testDataInput:testData[0:5]})
    print('p3=',p3.shape)#p3= (5, 500)
    print('p3[0,0]=',p3[0,0]) #130.451 knn distance p3[0,0]= 155.812，最核心的距离计算
    
    p4 = sess.run(f4,feed_dict={trainDataInput:trainData,testDataInput:testData[0:5]})#数据还是训练数据，测试数据
    print('p4=',p4.shape)
    print('p4[0,0]',p4[0,0])#-155.812，完成 对p3的取反
    
    # 同时查看两个参数
    p5,p6 = sess.run((f5,f6),feed_dict={trainDataInput:trainData,testDataInput:testData[0:5]})
    #p5= (5, 4) 每一张测试图片（5张）分别对应4张最近训练图片
    #p6= (5, 4)
    print('p5=',p5.shape)
    print('p6=',p6.shape)
    print('p5[0,0]',p5[0])# p5：-48.4275
    print('p6[0,0]',p6[0])# p6：314 index 下标
    
    p7 = sess.run(f7,feed_dict={trainDataInput:trainData,testDataInput:testData[0:5],trainLabelInput:trainLabel})
    print('p7=',p7.shape)# p7= (5, 4, 10)，p7一共分成5组，四行数据，每一行对应一个label标签
    print('p7[]',p7)
    
    p8 = sess.run(f8,feed_dict={trainDataInput:trainData,testDataInput:testData[0:5],trainLabelInput:trainLabel})
    print('p8=',p8.shape)
    print('p8[]=',p8)
    
    p9 = sess.run(f9,feed_dict={trainDataInput:trainData,testDataInput:testData[0:5],trainLabelInput:trainLabel})
    print('p9=',p9.shape)
    print('p9[]=',p9)
    
    p10 = np.argmax(testLabel[0:5],axis=1)
    print('p10[]=',p10)
j = 0
for i in range(0,5):
    if p10[i] == p9[i]: # 如果相同，认为是对的
        j = j+1
print('ac=',j*100/5) # 把最后的检测概率打印出来
    


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
trainData.shape= (500, 784)
trainLabel.shape= (500, 10)
testData.shape= (5, 784)
testLabel.shape= (5, 10)
testLabel= [[ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]]
p1= (5, 1, 784)
p2= (5, 500, 784)
p3= (5, 500)
p3[0,0]= 149.69
p4= (5, 500)
p4[0,0] -149.69
p5= (5, 4)
p6= (5, 4)
p5[0,0] [-107.67453003 -110.09804535 -111.78039551 -114.78039551]
p6[0,0] [228 426 370  96]
p7= (5, 4, 10)
p7[] [[[ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.]
  [ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]]

 [[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
  [ 1.  0.  0.  0.  0.  0